In [ ]:
import pandas as pd

#convert pandas series from German number format (1.000,5) to python number format (1000.5)
def make_column_numeric(se):
    se = se.str.replace('.','',regex=True)
    se = se.str.replace(',','.',regex=True)
    se = pd.to_numeric(se)
    return se

#INKAR data from bbsr
#https://www.inkar.de
df = pd.read_csv("data/inkar_commuting.csv",delimiter=";", header=[0,1])
df.columns = df.columns.map('_'.join)

#rename unnamed headers
df.rename({"Kennziffer_Unnamed: 0_level_1": "Kennziffer","Raumeinheit_Unnamed: 1_level_1":"Raumeinheit","Aggregat_Unnamed: 2_level_1":"Aggregated"}, axis=1,inplace=True)


years = ["2016","2017","2018","2019"]
for year in years:
    sv = "sozialversicherungspflichtig Beschäftigte am Wohnort_" + str(year)
    df[sv] = make_column_numeric(df[sv])

    
    df["Einpendler_" + str(year)] = make_column_numeric(df["Einpendler_" + str(year)])
    df["Auspendler_" + str(year)] = make_column_numeric(df["Auspendler_" + str(year)])
    df["Pendlersaldo_" + str(year)] = make_column_numeric(df["Pendlersaldo_" + str(year)])

    df["Pendler mit Arbeitsweg 50 km und mehr_" + str(year)] = make_column_numeric(df["Pendler mit Arbeitsweg 50 km und mehr_" + str(year)])
    df["Pendler mit Arbeitsweg 150 km und mehr_" + str(year)] = make_column_numeric(df["Pendler mit Arbeitsweg 150 km und mehr_" + str(year)])
    df["Pendler mit Arbeitsweg 300 km und mehr_" + str(year)] = make_column_numeric(df["Pendler mit Arbeitsweg 300 km und mehr_" + str(year)])

    df["pendler_50km_gesamt_" + str(year)] = df["Pendler mit Arbeitsweg 50 km und mehr_" + str(year)] *  df[sv]
    df["pendler_150km_gesamt_" + str(year)] = df["Pendler mit Arbeitsweg 150 km und mehr_" + str(year)] *  df[sv]
    df["pendler_300km_gesamt_" + str(year)] = df["Pendler mit Arbeitsweg 300 km und mehr_" + str(year)] *  df[sv]

df.rename({"Kennziffer":"AGS"},axis=1,inplace=True)

# Load geo coordinates of centroids of kreise (made with qgis)
geo = pd.read_csv("../general_data/kreise_geocoordinates.csv")
geo = geo.filter(['AGS','xcoord','ycoord','GEN'])

#merge on AGS
df = df.merge(geo, on="AGS",how="inner")

#save
df.to_csv("data/inkar_commuting_output.csv")
df



In [ ]:
hamburg = df.copy()
hamburg = hamburg[hamburg["AGS"].isin([2000,3359,3353,1053,1062,1060,1056])]
hamburg.to_csv("data/communiting_hamburg.csv")
hamburg